In [ ]:
import os
from tqdm import tqdm
import pandas as pd
import sys
import time
import logging
import numpy as np
import matplotlib.pyplot as plt
import cv2
import re
from sklearn.neighbors import NearestNeighbors
from pathlib import Path 

In [ ]:
def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(data, key=alphanum_key)

In [ ]:
def find_min_round(array):
    x = round(np.min(array[np.nonzero(array)]))
    return(x)

def find_max_round(array):
    x = round(np.max(array))
    return(x)

In [ ]:
openpose_path = '/Users/andreibirladeanu/Documents/Data/meal_openpose/1118_meal/'

In [ ]:
poses = [x for x in sorted_alphanumeric(os.listdir(openpose_path))]


In [ ]:
skeletons = []
for frame in sorted_alphanumeric(os.listdir(openpose_path)):
    if frame[0] != ".":
        data = pd.read_csv(openpose_path+frame)
        data = data.fillna(0)
    else:
        continue
    if (len(data.columns)-1)/3 == 1:
        skeletons.append({'skeleton1':[np.array(data['x_1']), np.array(data['y_1']), np.array(data['prob_1'])]})
    elif (len(data.columns)-1)/3 == 2:
        skeletons.append({'skeleton1':[np.array(data['x_1']), np.array(data['y_1']),  np.array(data['prob_1'])],'skeleton2':[np.array(data['x_2']), np.array(data['y_2']),
                                                                                                                             np.array(data['prob_2'])]})
    elif (len(data.columns)-1)/3 == 3:
        skeletons.append({'skeleton1':[np.array(data['x_1']), np.array(data['y_1']),  np.array(data['prob_1'])],'skeleton2':[np.array(data['x_2']), np.array(data['y_2']),
                             np.array(data['prob_2'])],
                          'skeleton3':[np.array(data['x_3']), np.array(data['y_3']),  np.array(data['prob_3'])]}) 

In [ ]:
### new pop rule
### extracting Skeleton 1

neigh = NearestNeighbors(n_neighbors=1, algorithm = 'brute', metric='euclidean')
new_skeleton1 = []
hooman1 = []
counter=0
hooman2_frames = []
main_kpt = 1
fallback = 0
frames_1=[]
thr = 20
for i in range(len(skeletons)): 
        if i==0:
            new_skeleton1.append(skeletons[i]['skeleton1'])
            frames_1.append(i+1)
            skeletons[i].pop('skeleton1')
            continue
        if len(skeletons[i])==1:
            if new_skeleton1[-1][0][main_kpt] and new_skeleton1[-1][1][main_kpt] and \
                skeletons[i]['skeleton1'][0][main_kpt] and  skeletons[i]['skeleton1'][1][main_kpt] > 0.0:
                neigh.fit([[new_skeleton1[-1][0][main_kpt],  new_skeleton1[-1][1][main_kpt]]])
                neigh.fit([[new_skeleton1[-1][0][main_kpt],  new_skeleton1[-1][1][main_kpt]]])
                dist, nn = neigh.kneighbors([[skeletons[i]['skeleton1'][0][main_kpt],  skeletons[i]['skeleton1'][1][main_kpt]]], 
                                        return_distance = True)
                
                if dist[0][0] < thr:
                    new_skeleton1.append(skeletons[i]['skeleton1'])
                    frames_1.append(i+1)
                    skeletons[i].pop('skeleton1')
            else:
                neigh.fit([[new_skeleton1[-1][0][fallback],  new_skeleton1[-1][1][fallback]]])
                neigh.fit([[new_skeleton1[-1][0][fallback],  new_skeleton1[-1][1][fallback]]])
                dist, nn = neigh.kneighbors([[skeletons[i]['skeleton1'][0][fallback],  skeletons[i]['skeleton1'][1][fallback]]], 
                                        return_distance = True)
                
                if dist[0][0] < thr:
                    new_skeleton1.append(skeletons[i]['skeleton1'])
                    frames_1.append(i+1)
                    skeletons[i].pop('skeleton1')
            
        elif len(skeletons[i])==2:
            if new_skeleton1[-1][0][main_kpt] and new_skeleton1[-1][1][main_kpt] and \
           skeletons[i]['skeleton1'][0][main_kpt] and  skeletons[i]['skeleton1'][1][main_kpt] and \
           skeletons[i]['skeleton2'][0][main_kpt] and  skeletons[i]['skeleton2'][1][main_kpt]> 0.0:
            
                neigh.fit([[new_skeleton1[-1][0][main_kpt],  new_skeleton1[-1][1][main_kpt]]])
                dist_0, nn_0 = neigh.kneighbors([[skeletons[i]['skeleton1'][0][main_kpt],  skeletons[i]['skeleton1'][1][main_kpt]]], 
                                        return_distance = True)
                dist_1, nn_1 = neigh.kneighbors([[skeletons[i]['skeleton2'][0][main_kpt],  skeletons[i]['skeleton2'][1][main_kpt]]], 
                                        return_distance = True)
                if dist_0 < dist_1 and dist_0 <thr:
                    new_skeleton1.append(skeletons[i]['skeleton1'])
                    frames_1.append(i+1)
                    skeletons[i].pop('skeleton1')
                elif dist_0 > dist_1 and dist_1 <thr:
                    new_skeleton1.append(skeletons[i]['skeleton2'])
                    frames_1.append(i+1)
                    skeletons[i].pop('skeleton2')
            else:
                neigh.fit([[new_skeleton1[-1][0][fallback],  new_skeleton1[-1][1][fallback]]])
                dist_0, nn_0 = neigh.kneighbors([[skeletons[i]['skeleton1'][0][fallback],  skeletons[i]['skeleton1'][1][fallback]]], 
                                        return_distance = True)
                dist_1, nn_1 = neigh.kneighbors([[skeletons[i]['skeleton2'][0][fallback],  skeletons[i]['skeleton2'][1][fallback]]], 
                                        return_distance = True)
                if dist_0 < dist_1 and dist_0 <thr:
                    new_skeleton1.append(skeletons[i]['skeleton1'])
                    frames_1.append(i+1)
                    skeletons[i].pop('skeleton1')
                elif dist_0 > dist_1 and dist_1 <thr:
                    new_skeleton1.append(skeletons[i]['skeleton2'])
                    frames_1.append(i+1)
                    skeletons[i].pop('skeleton2')
        
    
     
        elif len(skeletons[i])==3:
            if new_skeleton1[-1][0][main_kpt] and new_skeleton1[-1][1][main_kpt] and \
           skeletons[i]['skeleton1'][0][main_kpt] and  skeletons[i]['skeleton1'][1][main_kpt] and \
           skeletons[i]['skeleton2'][0][main_kpt] and  skeletons[i]['skeleton2'][1][main_kpt] and \
            skeletons[i]['skeleton3'][0][main_kpt] and  skeletons[i]['skeleton3'][1][main_kpt] > 0.0:
                neigh.fit([[new_skeleton1[-1][0][main_kpt],  new_skeleton1[-1][1][main_kpt]]])
                dist_0, nn_0 = neigh.kneighbors([[skeletons[i]['skeleton1'][0][main_kpt],  skeletons[i]['skeleton1'][1][main_kpt]]], 
                                        return_distance = True)
                dist_1, nn_1 = neigh.kneighbors([[skeletons[i]['skeleton2'][0][main_kpt],  skeletons[i]['skeleton2'][1][main_kpt]]], 
                                        return_distance = True)
                dist_2, nn_2 = neigh.kneighbors([[skeletons[i]['skeleton3'][0][main_kpt],  skeletons[i]['skeleton3'][1][main_kpt]]], 
                                        return_distance = True)
                if dist_0 < dist_1 < dist_2 and dist_0<thr:
                    new_skeleton1.append(skeletons[i]['skeleton1'])
                    frames_1.append(i+1)
                    skeletons[i].pop('skeleton1')
                elif dist_1 < dist_0 < dist_2 and dist_1<thr:
                    new_skeleton1.append(skeletons[i]['skeleton2'])
                    frames_1.append(i+1)
                    skeletons[i].pop('skeleton2')
                elif dist_2 < dist_1 < dist_0 and dist_2<thr:
                    new_skeleton1.append(skeletons[i]['skeleton3'])
                    frames_1.append(i+1)
                    skeletons[i].pop('skeleton3')
            else:
                neigh.fit([[new_skeleton1[-1][0][fallback],  new_skeleton1[-1][1][fallback]]])
                dist_0, nn_0 = neigh.kneighbors([[skeletons[i]['skeleton1'][0][fallback],  skeletons[i]['skeleton1'][1][fallback]]], 
                                        return_distance = True)
                dist_1, nn_1 = neigh.kneighbors([[skeletons[i]['skeleton2'][0][fallback],  skeletons[i]['skeleton2'][1][fallback]]], 
                                        return_distance = True)
                dist_2, nn_2 = neigh.kneighbors([[skeletons[i]['skeleton3'][0][fallback],  skeletons[i]['skeleton3'][1][fallback]]], 
                                        return_distance = True)
                if dist_0 < dist_1 < dist_2 and dist_0<thr:
                    new_skeleton1.append(skeletons[i]['skeleton1'])
                    frames_1.append(i+1)
                    skeletons[i].pop('skeleton1')
                elif dist_1 < dist_0 < dist_2 and dist_1<thr:
                    new_skeleton1.append(skeletons[i]['skeleton2'])
                    frames_1.append(i+1)
                    skeletons[i].pop('skeleton2')
                elif dist_2 < dist_1 < dist_0 and dist_2<thr:
                    new_skeleton1.append(skeletons[i]['skeleton3'])
                    frames_1.append(i+1)
                    skeletons[i].pop('skeleton3')

In [ ]:
### new pop rule
## extracting skeleton 2

neigh = NearestNeighbors(n_neighbors=1, algorithm = 'brute', metric='euclidean')
new_skeleton2 = []
hooman1 = []
counter=0
frames_2 = []
for i in range(len(skeletons)): 
    if skeletons[i] == {}:
        continue
    if len(skeletons[i])==1:
        new_skeleton2.append(list(skeletons[i].items())[0][1])
        frames_2.append(i+1)
        if list(skeletons[i].items())[0][0] == 'skeleton2':
            skeletons[i].pop('skeleton2')
        else:
            skeletons[i].pop('skeleton1')
    elif i==0 and len(skeletons[i])==2:
        new_skeleton2.append(list(skeletons[i].items())[0][1])
        frames_2.append(i+1)
        if list(skeletons[i].items())[0][0] == 'skeleton2':
            skeletons[i].pop('skeleton2')
        else:
            skeletons[i].pop('skeleton1')
        
    elif len(skeletons[i])==2 and len(skeletons[i-1])==1:

        if new_skeleton2[-1][0][main_kpt] and new_skeleton2[-1][1][main_kpt] and \
         skeletons[i][0][1][0][main_kpt] and skeletons[i][0][1][1][main_kpt] and \
         skeletons[i][1][1][0][main_kpt] and skeletons[i][1][1][1][main_kpt] > 0.0:
            neigh.fit([[new_skeleton2[-1][0][main_kpt],  new_skeleton2[-1][1][main_kpt]]])
            dist_0, nn_0 = neigh.kneighbors([[skeletons[i][0][1][0][main_kpt],  skeletons[i][0][1][1][main_kpt]]], 
                                    return_distance = True)
            dist_1, nn_1 = neigh.kneighbors([[skeletons[i][1][1][0][main_kpt],  skeletons[i][1][1][1][main_kpt]]], 
                                    return_distance = True)
            if dist_0 < dist_1 and dist_0 <thr:
                new_skeleton2.append(list(skeletons[i].items())[0][1])
                frames_2.append(i+1)
                if list(skeletons[i].items())[0][0] == 'skeleton2':
                    skeletons[i].pop('skeleton2')
                else:
                    skeletons[i].pop('skeleton1')
            elif dist_0 > dist_1 and dist_1 <thr:
                new_skeleton2.append(list(skeletons[i].items())[1][1])
                frames_2.append(i+1)
                if list(skeletons[i].items())[0][0] == 'skeleton2':
                    skeletons[i].pop('skeleton2')
                else:
                    skeletons[i].pop('skeleton1')
        else: 
            neigh.fit([[new_skeleton2[-1][0][fallback],  new_skeleton2[-1][1][fallback]]])
            dist_0, nn_0 = neigh.kneighbors([[skeletons[i][0][1][0][fallback],  skeletons[i][0][1][1][fallback]]], 
                                    return_distance = True)
            dist_1, nn_1 = neigh.kneighbors([[skeletons[i][1][1][0][fallback],  skeletons[i][1][1][1][fallback]]], 
                                    return_distance = True)
            if dist_0 < dist_1 and dist_0 <thr:
                new_skeleton2.append(list(skeletons[i].items())[0][1])
                frames_2.append(i+1)
                if list(skeletons[i].items())[0][0] == 'skeleton2':
                    skeletons[i].pop('skeleton2')
                else:
                    skeletons[i].pop('skeleton1')
            elif dist_0 > dist_1 and dist_1 <thr:
                new_skeleton2.append(list(skeletons[i].items())[1][1])
                frames_2.append(i+1)
                if list(skeletons[i].items())[0][0] == 'skeleton2':
                    skeletons[i].pop('skeleton2')
                else:
                    skeletons[i].pop('skeleton1')

        


In [ ]:
(list(skeletons[0].items())[0][1])

In [ ]:
skeletons[0]

In [ ]:
# skel 3 is what should be left 
new_skeleton3 = []
hooman1 = []
counter=0
frames_3 = []
for i in range(len(skeletons)): 
    if skeletons[i] == {}:
        continue
    if len(skeletons[i]) != {}:
        new_skeleton3.append(list(skeletons[i].items())[0][1])
        frames_3.append(i-1)

In [ ]:
cap = cv2.VideoCapture('/Users/andreibirladeanu/Documents/Data/meal_videos/1117_meal.mp4')
count = 0
image_1 = []
count2 = 0
while(cap.isOpened()):
    ret_val, image = cap.read()
    if ret_val == False:
                break
    count += 1
    if count ==1:
        image_1.append(image)
        #break
    
    if count in frames_1:
        
        cv2.imshow('human1', image[find_min_round(new_skeleton1[count2][1]):find_max_round(new_skeleton1[count2][1]),
                   find_min_round(new_skeleton1[count2][0]):find_max_round(new_skeleton1[count2][0])])
    if count in frames_2:
        cv2.imshow('human2', image[find_min_round(new_skeleton2[count2][1]):find_max_round(new_skeleton2[count2][1]),
                   find_min_round(new_skeleton2[count2][0]):find_max_round(new_skeleton2[count2][0])])
    if count in frames_3:
        cv2.imshow('human3', image[find_min_round(new_skeleton3[count2][1]):find_max_round(new_skeleton3[count2][1]),
                   find_min_round(new_skeleton3[count2][0]):find_max_round(new_skeleton3[count2][0])])    
    count2 +=1
    cv2.waitKey(0)

    #if count in frames_3:
     #   cv2.imshow('human3', image[find_min_round(new_skeleton3[count2][1]):find_max_round(new_skeleton3[count2][1]),
              #     find_min_round(new_skeleton3[count2][0]):find_max_round(new_skeleton3[count2][0])])
        
        
    
cv2.destroyAllWindows()

In [ ]:
len(new_skeleton1), len(new_skeleton2), len(new_skeleton3) 

In [ ]:
print('a')